In [1]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


## 1. Chrome 및 Selenium 설치 (필수!)

In [ ]:
# 최신 Chrome 및 ChromeDriver 설치
!apt-get update
!wget -q https://dl.google.com/linux/direct/google-chrome-stable_current_amd64.deb
!apt-get install -y ./google-chrome-stable_current_amd64.deb
!rm google-chrome-stable_current_amd64.deb

# Selenium 설치
!pip install -q selenium

# 설치 확인
!google-chrome --version
import selenium
print(f"Selenium 버전: {selenium.__version__}")

## 2. 라이브러리 임포트

In [3]:
import sys
import time
import os
import pandas as pd
from urllib.parse import quote
from selenium import webdriver
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from selenium.webdriver.common.by import By
from selenium.webdriver.common.keys import Keys
from selenium.webdriver.chrome.options import Options

## 3. Chrome 드라이버 설정 (2025년 Colab 호환 버전)

In [ ]:
def create_chrome_driver():
    """
    Colab 환경에 최적화된 Chrome WebDriver 생성 함수
    """
    chrome_options = Options()

    # 필수 옵션 (Colab headless 환경)
    chrome_options.add_argument('--headless')
    chrome_options.add_argument('--no-sandbox')
    chrome_options.add_argument('--disable-dev-shm-usage')

    # 추가 안정화 옵션
    chrome_options.add_argument('--disable-gpu')
    chrome_options.add_argument('--window-size=1920,1080')
    chrome_options.add_argument('--disable-extensions')
    chrome_options.add_argument('--disable-software-rasterizer')

    # 한국어 설정
    chrome_options.add_argument('lang=ko_KR')

    # User-Agent 설정 (YouTube 차단 방지)
    chrome_options.add_argument('user-agent=Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/120.0.0.0 Safari/537.36')

    # Selenium 4.x + 최신 Chrome = 자동 드라이버 관리
    driver = webdriver.Chrome(options=chrome_options)
    return driver

# 테스트 드라이버 생성
driver = create_chrome_driver()
print("✅ Chrome WebDriver가 성공적으로 생성되었습니다!")
driver.quit()

## 4. 유튜브 채널 내 영상 검색 후 url 수집 (byCSS_SELECTOR)

In [8]:
# 검색할 채널 & 키워드 설정
channel_name = "@NetflixKorea"
search_keyword = "이 사랑 통역되나요"

# 검색어를 URL 인코딩 (한글, 공백 처리)
encoded_keyword = quote(search_keyword)

# 유튜브 특정 채널 내 검색 URL 생성
url = f"https://www.youtube.com/{channel_name}/search?query={encoded_keyword}"

# 유튜브 검색 페이지 열기 (새 드라이버 생성)
driver = create_chrome_driver()

driver.get(url)
time.sleep(3)

# 페이지가 완전히 로드될 때까지 대기
WebDriverWait(driver, 10).until(EC.presence_of_element_located((By.TAG_NAME, "body")))

# 페이지 스크롤 다운 (Infinite Scroll)
last_height = driver.execute_script("return document.documentElement.scrollHeight")
scroll_count = 0
max_scrolls = 10  # 최대 스크롤 횟수 제한 (무한 루프 방지)

while scroll_count < max_scrolls:
    driver.execute_script("window.scrollTo(0, document.documentElement.scrollHeight);")
    time.sleep(3)
    new_height = driver.execute_script("return document.documentElement.scrollHeight")
    if new_height == last_height:
        break
    last_height = new_height
    scroll_count += 1

# 모든 영상 요소 가져오기 (한 번에 title과 url 수집)
elements = driver.find_elements(By.CSS_SELECTOR, 'a#video-title')

# title과 url을 동시에 수집 (길이 불일치 방지)
data = []
for element in elements:
    title = element.text
    url = element.get_attribute("href")
    if title and url:  # 둘 다 있는 경우만 추가
        data.append({'title': title, 'urls': url})

if data:
    print(f"테스트 URL: {data[0]['urls']}")
    print(f"수집된 영상 수: {len(data)}개")
else:
    print("❌ 영상 링크를 찾을 수 없습니다.")

# DataFrame 변환
results_df = pd.DataFrame(data)
results_df['video_id'] = results_df['urls'].apply(lambda x: x.split('v=')[1].split('&')[0] if x and 'v=' in x else None)

# 결측치 제거 (video_id가 없는 경우 데이터프레임에서 제거)
results_df = results_df.dropna(subset=['video_id']).reset_index(drop=True)

# 엑셀 파일 저장 (채널 검색용 파일명)
file_name = f"YouTube_Channel_{channel_name.replace('@', '')}_{search_keyword}.xlsx"
results_df.to_excel(file_name, index=False)

# 엑셀 파일 저장 (지정된 구글 드라이브 경로에 저장)
save_path = "/content/drive/MyDrive/mypython/data/"  # 본인 구글드라이브의 작업 폴더 경로에 'data' 폴더 만들기
full_path = save_path + file_name  # 전체 파일 경로 생성

# 폴더가 존재하는지 확인 후 저장
if os.path.exists(save_path):
    results_df.to_excel(full_path, index=False)  # 엑셀 파일 저장
    print(f"📁 구글 드라이브에 저장 완료: {full_path}")
else:
    print(f"⚠️ 경로가 존재하지 않습니다: {save_path}")
    print("현재 디렉토리에만 저장됩니다.")

# ChromeDriver 종료
driver.quit()

print(f"🎉 크롤링 완료! 데이터가 '{file_name}'로 저장되었습니다.")
print(f"📊 최종 수집된 영상 수: {len(results_df)}개")

# Colab에서 파일 다운로드
from google.colab import files
files.download(file_name)

테스트 URL: https://www.youtube.com/watch?v=x6-vWD0RpRE
수집된 영상 수: 53개
📁 구글 드라이브에 저장 완료: /content/drive/MyDrive/mypython/data/YouTube_Channel_NetflixKorea_이 사랑 통역되나요.xlsx
🎉 크롤링 완료! 데이터가 'YouTube_Channel_NetflixKorea_이 사랑 통역되나요.xlsx'로 저장되었습니다.
📊 최종 수집된 영상 수: 53개


<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

## 5. 유튜브 전체 영상 검색 url 수집 (byCSS_SELECTOR)

In [9]:
# 검색할 키워드 설정
search_keyword = "이 사랑 통역되나요"

# 검색어를 URL 인코딩 (한글, 공백 처리)
encoded_keyword = quote(search_keyword)

# YouTube 전체 검색 URL 생성
url = f"https://www.youtube.com/results?search_query={encoded_keyword}"

# 유튜브 검색 페이지 열기 (새 드라이버 생성)
driver = create_chrome_driver()

driver.get(url)
time.sleep(2)

# 페이지가 완전히 로드될 때까지 대기
WebDriverWait(driver, 10).until(EC.presence_of_element_located((By.TAG_NAME, "body")))

# 페이지 스크롤 다운 (Infinite Scroll)
last_height = driver.execute_script("return document.documentElement.scrollHeight")
scroll_count = 0
max_scrolls = 10  # 최대 스크롤 횟수 제한 (무한 루프 방지)

while scroll_count < max_scrolls:
    driver.execute_script("window.scrollTo(0, document.documentElement.scrollHeight);")
    time.sleep(2)
    new_height = driver.execute_script("return document.documentElement.scrollHeight")
    if new_height == last_height:
        break
    last_height = new_height
    scroll_count += 1

# 모든 영상 요소 가져오기 (한 번에 title과 url 수집)
elements = driver.find_elements(By.CSS_SELECTOR, 'a#video-title')

# title과 url을 동시에 수집 (길이 불일치 방지)
data = []
for element in elements:
    title = element.text
    url = element.get_attribute("href")
    if title and url:  # 둘 다 있는 경우만 추가
        data.append({'title': title, 'urls': url})

if data:
    print(f"테스트 URL: {data[0]['urls']}")
    print(f"수집된 영상 수: {len(data)}개")
else:
    print("❌ 영상 링크를 찾을 수 없습니다.")

# DataFrame 변환
results_df = pd.DataFrame(data)
results_df['video_id'] = results_df['urls'].apply(lambda x: x.split('v=')[1].split('&')[0] if x and 'v=' in x else None)

# 결측치 제거 (video_id가 없는 경우 데이터프레임에서 제거)
results_df = results_df.dropna(subset=['video_id']).reset_index(drop=True)

# 엑셀 파일 저장 (전체 검색용 파일명)
file_name = f"YouTube_Search_{search_keyword}.xlsx"
results_df.to_excel(file_name, index=False)

# 엑셀 파일 저장 (지정된 구글 드라이브 경로에 저장)
save_path = "/content/drive/MyDrive/mypython/data/"
full_path = save_path + file_name  # 전체 파일 경로 생성

# 폴더가 존재하는지 확인 후 저장
if os.path.exists(save_path):
    results_df.to_excel(full_path, index=False)  # 엑셀 파일 저장
    print(f"📁 구글 드라이브에 저장 완료: {full_path}")
else:
    print(f"⚠️ 경로가 존재하지 않습니다: {save_path}")
    print("현재 디렉토리에만 저장됩니다.")

# ChromeDriver 종료
driver.quit()

print(f"🎉 크롤링 완료! 데이터가 '{file_name}'로 저장되었습니다.")
print(f"📊 최종 수집된 영상 수: {len(results_df)}개")

# Colab에서 파일 다운로드
from google.colab import files
files.download(file_name)

테스트 URL: https://www.youtube.com/watch?v=IIHQ1Z2yu80&pp=ygUa7J20IOyCrOuekSDthrXsl63rkJjrgpjsmpTSBwkJhwoBhyohjO8%3D
수집된 영상 수: 180개
📁 구글 드라이브에 저장 완료: /content/drive/MyDrive/mypython/data/YouTube_Search_이 사랑 통역되나요.xlsx
🎉 크롤링 완료! 데이터가 'YouTube_Search_이 사랑 통역되나요.xlsx'로 저장되었습니다.
📊 최종 수집된 영상 수: 79개


<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

---
## 문제 해결 가이드

### 에러가 발생할 경우

**1. SessionNotCreatedException 에러**
- 런타임 재시작: 메뉴 → 런타임 → 런타임 다시 시작
- 1번 셀(Chrome 설치)부터 다시 실행

**2. 영상이 검색되지 않는 경우**
- YouTube가 봇을 차단할 수 있음
- time.sleep() 시간을 늘려보기 (3초 → 5초)
- 검색 키워드를 변경하여 테스트